In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import gzip
import json

In [3]:
raw_data = pd.read_csv("Data/Gift_Cards.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [4]:
data

,User,Item,Rating
5585,A332OLZRFWTD7X,B004KNWWU4,5.0
17692,A3TVWR2M1IZY29,B004LLILGO,5.0
63947,A3LZ0QBF79IQW3,B00AF0K90Q,5.0
124065,A1HG4ZW9BAGPOE,B014S24DAI,5.0
35190,A1DIHNGRGJY6Q7,B005EISPOG,5.0
...,...,...,...
108969,AJ7HMMQYY00HA,B00MV9FCFQ,5.0
10945,A4D4TXCFRFM3O,B004LLIL0A,4.0
37315,A1BKINP38124K7,B005ESMJ02,5.0
83294,A2MH59P0AFCUOD,B00CHSWG6O,5.0


In [5]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,B001GXRQW0,B002BSHDJK,B002DN7XS4,B002NZXF9S,B002O018DM,B002QFXC7U,B002QTM0Y2,B002UKLD7M,B002VG4AR0,B002W8YL6W,...,B01GF6WRRG,B01GF6X9UU,B01GF7GNCA,B01GKWEH64,B01GKWEPBG,B01GKZ37SA,B01GKZ3SQG,B01GOQIKRI,B01GP1W4LA,B01H5PPJT4
User,,,,,,,,,,,,,,,,,,,,,
A0622647I26JXAXSUPER,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A09137477D4AG47IV249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100DAK0H0VGXZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100S8O2ADCUXK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100T8SHG8HVKK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = ratings_matrix.T
X1 = X
X.head()

User,A0622647I26JXAXSUPER,A09137477D4AG47IV249,A100DAK0H0VGXZ,A100S8O2ADCUXK,A100T8SHG8HVKK,A101U0GCJYIAZ6,A101WFNV9RNIHU,A102RJAPQX005I,A1047EDJ84IMAS,A104J6V22YLTJ,...,AZWC78IPHX6HX,AZWIPMF2QYFX6,AZWPJYXRWSJYY,AZWSK0PW0JQM4,AZXLPHKI6Z0LB,AZY5EY0B860SN,AZY5WTW6SEMGY,AZY9K2FIDN8N8,AZZD30PYJVGI7,AZZJOT0K986S4
Item,,,,,,,,,,,,,,,,,,,,,
B001GXRQW0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002BSHDJK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002DN7XS4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002NZXF9S,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002O018DM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(878, 10)

In [8]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(878, 878)

In [9]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

805

In [10]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(878,)

In [11]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]

['B001GXRQW0',
 'B002QTM0Y2',
 'B003CMYYGY',
 'B004KNWWOA',
 'B004KNWWUY',
 'B004KNWX3U',
 'B004KNWX4Y',
 'B004KNWX5I',
 'B004KNWX6C',
 'B004KNWX6M',
 'B004LLIKZ6',
 'B004LLIL0K',
 'B004LLIL78',
 'B004LLILIM',
 'B004LLILKK',
 'B004LLJ65O',
 'B004LLJ6AE',
 'B004RD8XPK',
 'B004RD8XQE',
 'B004RD967Y',
 'B004W8D102',
 'B004WKPVBC',
 'B005EISOSS',
 'B005EISPCS']

In [12]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_Gift_Cards.json.gz')
data_set = review_data[:30000]
data_set
    

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Gift Cards, Gift Cards]",,[Gift card for the purchase of goods or servic...,,Serendipity 3 $100.00 Gift Card,[],[],,Serendipity 3,[],[],[],{' Product Dimensions: ': '3.4 x 2.1 ...,Grocery,,,,B001BKEWF2
1,"[Gift Cards, Gift Cards]",,[Amazon.com Gift Cards are the perfect way to ...,,Amazon.com Gift Cards,[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,[Amazon.com Gift cards never expire and carry ...,[],"[BT00DC6QU4, B01I4AHZXC, B0719C5P56, B01K8RLHZ...","{'Shipping Weight:': '0.5 ounces', 'Domestic S...",Gift Cards,,,,B001GXRQW0
2,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""> <br>Amazon.com Gift Cards...",,"Amazon.com Gift Cards, Pack of 50 (Old Version...","[B005ESMEBQ, B007EE5WNA, B007EE5OBU, B07HJHK8Y...",[https://images-na.ssl-images-amazon.com/image...,,Amazon,"[Contains 50 Gift Cards, Gift Card has no fees...","1,786 in Gift Cards (","[B005ESMEBQ, B007EE5OBU, B007EE5WNA, B007EE60N...","{'Shipping Weight:': '1.8 pounds (', 'Domestic...",Gift Cards,,,"$1,250.00",B001H53QE4
3,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""> <h4>Amazon.com Gift Cards...",,"Amazon.com $50 Gift Cards, Pack of 50 (Old Ve...",[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,"[Contains 50 Gift Cards, Gift Card has no fees...",[],[],"{'Shipping Weight:': '1.6 pounds', 'Domestic S...",Gift Cards,,,,B001H53QEO
4,"[Gift Cards, Gift Cards]",,[],,Wood Puzzle Magic Box Gift Card,[],[https://images-na.ssl-images-amazon.com/image...,,Creative Crafthouse,[],"[>#2,893,775 in Toys & Games (See Top 100 in T...",[],{},Toys & Games,,,,B001KMWN2K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,"[Gift Cards, Gift Cards]",,"[Hand-Pattied Burgers Chicken, Steaks & Fish ...",,Harvey&#39;s Grill and Bar &amp; Hayden&#39;s...,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"5,808 in Gift Cards (",[],"{'Domestic Shipping: ': 'Currently, item can b...",Gift Cards,,,,B01GMFJ8JU
1543,"[Gift Cards, Gift Cards]",,[<strong>Smoothie King Gift Cards. The Perfect...,,Smoothie King Gift Card,"[B00NVUDIZ0, B00BXLVE6Y, B00O4I5S7O, B00GOLGWV...",[https://images-na.ssl-images-amazon.com/image...,,QuickGifts,"[Over 500 Locations Nationwide!, Smoothies, He...","1,201 in Gift Cards (","[B00NVUDIZ0, B00GOLGWVK, B07JKD8G5X, B00BXLVE6...",{'ASIN:': 'B01GOQIKRI'},Gift Cards,,,$10.00,B01GOQIKRI
1544,"[Gift Cards, Gift Cards]",,"[Cattle Baron Restaurants, Inc.is a privately ...",,Cattle Baron Restaurant Gift Card,"[B00CXZPG0O, B00PG84W32]",[https://images-na.ssl-images-amazon.com/image...,,QuickGifts,"[9 Locations over New Mexico and Texas!, Join ...","3,302 in Gift Cards (",[],{'ASIN:': 'B01GOQHGL4'},Gift Cards,,,,B01GOQHGL4
1545,"[Gift Cards, Gift Cards]",,"[The cards, available in $25 increments up to ...",,Go Play Golf Gift Card,"[B077CZY68Z, B07JJHVWXL, B00NVUDIZ0, B01CZ54L6...",[],,Go Play Golf by Fairway Rewards,[Go Play Golf - the only national gift card th...,"1,015 in Gift Cards (","[B01LYIR4CZ, B077CZY68Z, B07736DNZB, B01N9AE2Z...","{'Shipping Weight:': '0.6 ounces (', 'ASIN:': ...",Gift Cards,,,$50.00,B01GP1W4LA


In [13]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})

In [14]:
top25

,ID
0,B001GXRQW0
1,B002QTM0Y2
2,B003CMYYGY
3,B004KNWWOA
4,B004KNWWUY
5,B004KNWX3U
6,B004KNWX4Y
7,B004KNWX5I
8,B004KNWX6C
9,B004KNWX6M


In [15]:
top25.merge(data_set,left_on = 'ID', right_on = 'asin')

,ID,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,B001GXRQW0,"[Gift Cards, Gift Cards]",,[Amazon.com Gift Cards are the perfect way to ...,,Amazon.com Gift Cards,[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,[Amazon.com Gift cards never expire and carry ...,[],"[BT00DC6QU4, B01I4AHZXC, B0719C5P56, B01K8RLHZ...","{'Shipping Weight:': '0.5 ounces', 'Domestic S...",Gift Cards,,,,B001GXRQW0
1,B002QTM0Y2,"[Gift Cards, Gift Cards]",,"[With more than 40,000 products to choose from...",,The Home Depot Crate Gift Card,[],[],,The Home Depot,[Premium greeting cards options are available....,"3,925 in Gift Cards (",[],{'ASIN:': 'B002QTM0Y2'},Gift Cards,,,,B002QTM0Y2
2,B003CMYYGY,"[Gift Cards, Gift Cards]",,[],,Jack in the Box Gift Card Collection,[],[],,Jack in the Box,[Premium greeting cards options are available....,"4,519 in Gift Cards (",[],{'ASIN:': 'B003CMYYGY'},Gift Cards,,,,B003CMYYGY
3,B004KNWWOA,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Graduation (Congra...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWOA
4,B004KNWWUY,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Amazon Books,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWUY
5,B004KNWX3U,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Amazon Kindle,[],[],,,[],[],"[BT00DDC7BK, BT00DC6QU4, B00X4SHPFS, B0091JKPP...",{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX3U
6,B004KNWX4Y,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Mother&#39;s...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX4Y
7,B004KNWX5I,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Amazon Music (Notes),[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX5I
8,B004KNWX6C,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Celebrate,[],[],,,[],[],"[BT00DDC7BK, BT00DC6QU4, B06ZY43PDR, B0719C5P56]",{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX6C
9,B004KNWX6M,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - New House,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX6M


In [16]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')

In [19]:
Top25Recommend[Top25Recommend.columns[:2]]
Top25Recommend = Top25Recommend[Top25Recommend.columns[5:6]]

In [20]:
Top25Recommend

,title
0,Amazon.com Gift Cards
1,The Home Depot Crate Gift Card
2,Jack in the Box Gift Card Collection
3,Amazon Gift Card - Print - Graduation (Congra...
4,Amazon Gift Card - Print - Amazon Books
5,Amazon Gift Card - Print - Amazon Kindle
6,Amazon Gift Card - Print - Happy Mother&#39;s...
7,Amazon Gift Card - Print - Amazon Music (Notes)
8,Amazon Gift Card - Print - Celebrate
9,Amazon Gift Card - Print - New House
